In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, DoubleType
import sys
spark = SparkSession.builder.appName("SilverTable").getOrCreate()
source_schema_name = "_bronze"
target_schema_name = "_silver"




In [ ]:
# fixing module import errors
import os
__file__ = "02.enrich customer & product table.ipynb"

script_dir = os.path.dirname(os.path.abspath(__file__))
print(f'this is script dir {script_dir}')
root_dir = os.path.normpath(os.path.join(os.path.dirname(os.path.abspath(__file__)), '..'))
print(f'this is root dir {root_dir}')
sys.path.insert(0,root_dir)
print(sys.path)

Customer Enriched Table

In [ ]:
from pipeline.transformation.customer_utils import fix_customer_name_string,fix_phone_number_string
from pipeline.transformation.common_utils import standardize_column_names

# Define a UDF 
udfFixCustomerName = F.udf(fix_customer_name_string,StringType())
udfFixPhoneNumber = F.udf(fix_phone_number_string,StringType())


In [ ]:

table_name = "customer_table"

customer_df = spark.sql(f"SELECT * FROM {source_schema_name}.{table_name}")


customer_df = customer_df.withColumn("Customer Name", udfFixCustomerName(F.col("Customer Name")))
customer_df = customer_df.withColumn("Phone", udfFixPhoneNumber(F.col("Phone")))

customer_df = standardize_column_names(customer_df)
customer_df.write.mode("overwrite").saveAsTable(f"{target_schema_name}.{table_name}")

Product Enriched Table

In [ ]:
from pipeline.transformation.products_utils import clean_product_name

udfFixProductName = F.udf(clean_product_name,StringType())

table_name = 'product_table'
product_df = spark.sql(f"SELECT * FROM {source_schema_name}.{table_name}")
product_df = standardize_column_names(product_df)
product_df = product_df.withColumn("Product_Name",udfFixProductName("Product_Name"))
product_df.write.mode("overwrite").saveAsTable(f"{target_schema_name}.{table_name}")


Orders Table:

In [ ]:
from pipeline.transformation.common_utils import transform_date
table_name = 'orders_table'
orders_df = spark.sql(f"SELECT * FROM {source_schema_name}.{table_name}")
orders_df = standardize_column_names(orders_df)
orders_df = transform_date(orders_df, 'Order_Date')
orders_df = transform_date(orders_df,'Ship_Date')
orders_df.write.mode("overwrite").saveAsTable(f"{target_schema_name}.{table_name}")